# Lane Line Detection

if the min and max value of canny increase the edges in the image decrease the focused on the important deges

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math

img = cv2.imread("C://Users//HP//Downloads//solidWhiteCurve (1).jpg")
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
blur = cv2.GaussianBlur(gray, (3, 3), 0)
edges1 = cv2.Canny(blur, 100, 200)

img2 = cv2.imread("C://Users//HP//Downloads//solidWhiteCurve (1).jpg")
image2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
gray2 = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)
blur2 = cv2.GaussianBlur(gray2, (3, 3), 0)
edges2 = cv2.Canny(blur2, 100, 200)



cv2.namedWindow('my_image')

def nothing(x):
    pass

cv2.createTrackbar('x','my_image',0,500,nothing)
cv2.createTrackbar('y','my_image',0,500,nothing)

cv2.createTrackbar('x2','my_image',0,500,nothing)
cv2.createTrackbar('y2','my_image',0,500,nothing)



while True :
    x=cv2.getTrackbarPos('x','my_image')
    y=cv2.getTrackbarPos('y','my_image')
    x2=cv2.getTrackbarPos('x2','my_image')
    y2=cv2.getTrackbarPos('y2','my_image')

    gray = cv2.imread("C://Users//HP//Downloads//solidWhiteCurve (1).jpg", 0)
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    edges1 = cv2.Canny(blur, x, y)
    
    gray2 = cv2.imread("C://Users//HP//Downloads//solidYellowCurve2 (1).jpg", 0)
    blur2 = cv2.GaussianBlur(gray, (3, 3), 0)
    edges2 = cv2.Canny(blur2, x2, y2)


    

    cv2.imshow('my_image',edges1)
    cv2.imshow('my_image2',edges2)

    k=cv2.waitKey(1)
    if k==ord('q'):
        break
cv2.destroyAllWindows()



# Helpers Function
to hard to detect the lines

In [2]:
#helpers function

def helpers_edges(gray):
    kernel_size = 5 #kernel size for Gaussian smoothing / blurring
    blur_gray= cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)#Gaussian smoothing / blurring

    ##
    # finding edges - Canny Edge detection (strong gradient between adjacent pixels)
    threshold1 = 50 # 50
    threshold2 = 150 #  180
    edges = cv2.Canny(blur_gray, threshold1, threshold2)
    return(edges)


def helpers_masked_edges(edges):
    mask = np.zeros_like(edges)
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(edges.shape) > 2:
        channel_count = edges.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    imshape = edges.shape        
    vertices = np.array([[((1/6*imshape[1]),imshape[0]),((5/12*imshape[1]), (3/5*imshape[0])), ((7/12*imshape[1]), (3/5*imshape[0])), ((9/10*imshape[1]),imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)
    return(masked_edges)

def helpers_formulate_lanes(lines, img, color=[255, 0, 0], thickness=2):    
    #algorithm logic:
    #aim is to find x_min, y_min, x_max, y_max , slope and intercept for both lanes lines.
    #for each line returned from the hough lines function:
    #   calculate slope
    #   calculate intercept
    #   store positive and negative slope and intercept values separately in arrays.
    #   y_min is the minimum of all the y coordinates.
    #   y_max is the bottom of the image from where the lane lines start.
    #   slope and intercept values for both lines are just the averages of all values stored previously.
    #  x_min and x_max can now be calculated by fitting all the lines in the equation x = (y - intercept)/slope.
    
    #LINE DISPLAY PARAMETERS
    color = [243, 105, 14]
    thickness = 12
    
    #LINE PARAMETERS
    SLOPE_THRESHOLD = 0.3
    Y_MIN_ADJUST = 15
    
    positive_slopes = []
    negative_slopes = []
    
    positive_intercepts = []
    negative_intercepts = []
    
    #named as y_max despte being at the bottom corner of the image due to y axis in reverse direction
    y_max = img.shape[0]
    y_min = img.shape[0]
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            
            #calculate slope for the line
            slope = (y2-y1)/(x2-x1)
            intercept = y2 - (slope*x2)
            
            #for negative slope
            if slope < 0.0 and slope > -math.inf and abs(slope) > SLOPE_THRESHOLD:
                #print('negative slope')
                negative_slopes.append(slope)
                negative_intercepts.append(intercept)
                
            #for positive slope
            elif slope > 0.0 and slope < math.inf and abs(slope) > SLOPE_THRESHOLD:
                #print('positive slope')
                positive_slopes.append(slope)
                positive_intercepts.append(intercept)
            
            y_min = min(y_min, y1, y2)
            #cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    
    y_min+=Y_MIN_ADJUST
    
    #get averages for positive and negative slopes
    positive_slope_mean = np.mean(positive_slopes)
    negative_slope_mean = np.mean(negative_slopes)

    #get averages for potitive and negative intercepts
    positive_intercept_mean = np.mean(positive_intercepts)
    negative_intercept_mean = np.mean(negative_intercepts)
    lst = [
        [[0, 0, 0, 0]],
        [[0, 0, 0, 0]]
    ]

    #calculation of coordinates for lane for positive slopes
    if len(positive_slopes) > 0:
        x_max = int((y_max - positive_intercept_mean)/positive_slope_mean)
        x_min = int((y_min - positive_intercept_mean)/positive_slope_mean)
        #cv2.line(img, (x_min, y_min), (x_max, y_max), color, thickness)
        lst[0][0] = [x_min, y_min, x_max, y_max]
    
    #calculation of coordinates for lane for negative slopes
    if len(negative_slopes) > 0:
        x_max = int((y_max - negative_intercept_mean)/negative_slope_mean)
        x_min = int((y_min - negative_intercept_mean)/negative_slope_mean)
        #cv2.line(img, (x_min, y_min), (x_max, y_max), color, thickness)
        lst[1][0] = [x_min, y_min, x_max, y_max]
        #lst.append([x_min, y_min, y_min, y_max])
    return(np.array(lst))

def helpers_draw_lines(lines, masked_edges):
    color = [243, 105, 14]
    thickness = 12
    lines_image = np.zeros((masked_edges.shape[0], masked_edges.shape[1], 3), dtype=np.uint8)
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(lines_image, (x1, y1), (x2, y2), color, thickness)
    return(lines_image)

#HOUGH LINES PARAMETERS
RHO = 3.5 
THETA = np.pi/180
MIN_VOTES = 30    
MIN_LINE_LEN = 5 
MAX_LINE_GAP= 25  

def helpers_hough_lines(img):
    """
    `img` should be the output of a Canny transform.
        
    Returns the lines from hough transform
    """
    lines = cv2.HoughLinesP(img, RHO, THETA, MIN_VOTES, np.array([]), minLineLength=MIN_LINE_LEN, maxLineGap=MAX_LINE_GAP)
    return lines

RHO = 1                 # try: 1 - 4 (0.5 increments)  
THETA = np.pi/180       # Usually this is Ok
MIN_VOTES = 10          # try: 10 - 50                  # Typical: 30
MIN_LINE_LEN = 5 
MAX_LINE_GAP = 25 

def hough_lines(img):
    lines = cv2.HoughLinesP(img, RHO, THETA, MIN_VOTES, np.array([]), minLineLength=MIN_LINE_LEN, maxLineGap=MAX_LINE_GAP)
    return lines

In [3]:
masked_1=helpers_masked_edges(edges1)
masked_2=helpers_masked_edges(edges2)

# Detect lane line on image

In [4]:
RHO = 1                 
THETA = np.pi/180       
MIN_VOTES = 10               
MIN_LINE_LEN = 5 
MAX_LINE_GAP = 25 

lines = cv2.HoughLinesP(masked_1, RHO, THETA, MIN_VOTES, np.array([]), minLineLength=MIN_LINE_LEN, maxLineGap=MAX_LINE_GAP)




cv2.namedWindow('my_image')

def nothing(x):
    pass

cv2.createTrackbar('x','my_image',10,50,nothing)
cv2.createTrackbar('y','my_image',0,30,nothing)
cv2.createTrackbar('z','my_image',0,50,nothing)



while True :
    x=cv2.getTrackbarPos('x','my_image')
    y=cv2.getTrackbarPos('y','my_image')
    z=cv2.getTrackbarPos('z','my_image')

    
    ROI_edges_img1 = helpers_masked_edges(masked_1)
    lines1 = cv2.HoughLinesP(ROI_edges_img1, RHO, THETA, x, np.array([]), minLineLength=y, maxLineGap=z)
    hough_lines_image = helpers_draw_lines(lines, ROI_edges_img1)
    lanes1 = helpers_formulate_lanes(lines1, ROI_edges_img1)
    lanes_image1 = helpers_draw_lines(lanes1, ROI_edges_img1)
    final_image1 = cv2.addWeighted(img, 0.8, lanes_image1, 1, 0) 



    ROI_edges_img2 = helpers_masked_edges(masked_2)
    lines2 = cv2.HoughLinesP(ROI_edges_img2, RHO, THETA, x, np.array([]), minLineLength=y, maxLineGap=z)
    hough_lines_image2 = helpers_draw_lines(lines2, ROI_edges_img2)
    lanes2 = helpers_formulate_lanes(lines2, ROI_edges_img2)
    lanes_image2 = helpers_draw_lines(lanes2, ROI_edges_img2)
    final_image2 = cv2.addWeighted(img2, 0.8, lanes_image2, 1, 0) 

    cv2.imshow('my_image',final_image1)
    cv2.imshow('my_image2',final_image2)

    k=cv2.waitKey(1)
    if k==ord('q'):
        break
cv2.destroyAllWindows()



<ipython-input-2-19c9901807ce>:63: RuntimeWarning: invalid value encountered in int_scalars
  slope = (y2-y1)/(x2-x1)
<ipython-input-2-19c9901807ce>:63: RuntimeWarning: divide by zero encountered in int_scalars
  slope = (y2-y1)/(x2-x1)


# Detect Lane Line on Real-Time Video

In [5]:
video = cv2.VideoCapture("C://Users//HP//Downloads//Video//solidWhiteRight.mp4")

if not video.isOpened():
    print("Error opening video file.")

while video.isOpened():
    ret, frame = video.read()

    if not ret:
        break

        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    edges = cv2.Canny(blur, 100, 200)
    ROI_edges_img = helpers_masked_edges(edges)
    lines = hough_lines(ROI_edges_img)
    hough_lines_image = helpers_draw_lines(lines, ROI_edges_img)
    lanes = helpers_formulate_lanes(lines, ROI_edges_img)
    lanes_image = helpers_draw_lines(lanes, ROI_edges_img)
    final_image = cv2.addWeighted(frame, 0.8, lanes_image, 1, 0) 





    cv2.imshow("Video", final_image)

    delay = int(1000 / video.get(cv2.CAP_PROP_FPS) / 2)
    if cv2.waitKey(delay) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()